In [13]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer, TopicPartition
import redis
import time

offset = "latest"

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron:
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        """Wait for activation message from Kafka before processing"""
        topic = f'layer-{self.layer_id[-1]}'
        consumer = KafkaConsumer(
            bootstrap_servers='kafka:9092',
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset=offset,  # ✅ Always read new messages
            enable_auto_commit=True,
            group_id=f'group-layer-{self.layer_id}',
            consumer_timeout_ms=120000
        )

        partition = TopicPartition(topic, self.neuron_id)
        consumer.assign([partition])

        for message in consumer:
            print(f"✅ Neuron {self.neuron_id} received message: {message.value}")
            if 'layer' in message.value and message.value['layer'] == self.layer_id:
                print(f"🚀 Neuron {self.neuron_id} in {self.layer_id} activated!")
                break

        consumer.close()

        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        """Trigger neuron activations via Kafka and activate the next layer when finished"""
        producer = KafkaProducer(bootstrap_servers='kafka:9092',
                                 value_serializer=lambda v: json.dumps(v).encode('utf-8'),
                                 retries=5, request_timeout_ms=10000)
    
        topic = f'layer-{self.layer_id[-1]}'
        activation_message = {'layer': self.layer_id}

        print(f"📤 Layer {self.layer_id} sending activation messages to topic {topic}...")

        for neuron_id in range(len(self.neurons)):
            producer.send(topic, key=str(neuron_id).encode(), value=activation_message, partition=neuron_id)
            #print(f"✅ Message sent to {topic}, partition {neuron_id}")

        producer.flush()
        producer.close()

        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)
        
        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())
        
        # Activate the next layer
        producer = KafkaProducer(bootstrap_servers='kafka:9092',
                                 value_serializer=lambda v: json.dumps(v).encode('utf-8'))
        producer.send('activate-layer', {'layer': f'layer-{int(self.layer_id[-1]) + 1}'} if not self.is_final_layer else {'layer': 'final'})
        producer.flush()
        producer.close()
    
        return outputs

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Build network
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                layer_id=layer_name,
                neuron_id=idx,
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for idx, node in enumerate(layer_info['nodes'])
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

# Forward pass for single image
def forward_pass(layers, input_data, image_id):
    producer = KafkaProducer(bootstrap_servers='kafka:9092',
                             value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    print("🔥 Sending initial activation message to layer-0...")
    producer.send('layer-0', {'layer': 'layer_0'})
    producer.flush()
    producer.close()
    
    for layer in layers:
        input_data = layer.forward(input_data)
    
    prediction = int(np.argmax(input_data))
    redis_client.hset('predictions', image_id, prediction)  # Store all predictions
    return prediction

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    prediction = forward_pass(network, image_np, i)
    print(f"Image {i} Prediction: {prediction}, Label: {label}")

# Calculate accuracy
predictions = redis_client.hgetall('predictions')
correct = sum(int(predictions[k]) == mnist_test[int(k)][1] for k in predictions)
accuracy = correct / len(predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(offset)

🔥 Sending initial activation message to layer-0...
📤 Layer layer_0 sending activation messages to topic layer-0...


KeyboardInterrupt: 

In [1]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer, TopicPartition
from pcomp.redis_utils import RedisHandler
import uuid
import time

# settings
KAFKA_BROKER = 'kafka:9092'
REDIS_HOST = 'host.docker.internal'
REDIS_PORT = 6379
OFFSET_RESET = "latest"

redis_handler = RedisHandler(REDIS_HOST, REDIS_PORT, 0)

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

ACTIVATIONS = {"relu": relu, "softmax": softmax}

def generate_group_id():
    return str(uuid.uuid4())

class Neuron:
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        topic = f'layer-{self.layer_id[-1]}'
        consumer = KafkaConsumer(
            bootstrap_servers=KAFKA_BROKER,
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset=OFFSET_RESET,
            enable_auto_commit=True,
            group_id=generate_group_id(),
            consumer_timeout_ms=120000
        )

        partition = TopicPartition(topic, self.neuron_id)
        consumer.assign([partition])

        print(f"Neuron {self.neuron_id} in {self.layer_id} waiting for activation...")

        for message in consumer:
            if message.value.get('layer') == self.layer_id:
                print(f"Neuron {self.neuron_id} in {self.layer_id} activated with message: {message.value}")
                break

        consumer.close()

        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neurons, is_final_layer=False):
        self.layer_id = layer_id
        self.neurons = neurons
        self.is_final_layer = is_final_layer

    def forward(self, input_data):
        consumer = KafkaConsumer(
            'activate-layer',
            bootstrap_servers=KAFKA_BROKER,
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset=OFFSET_RESET,
            enable_auto_commit=False,
            group_id=generate_group_id(),
            consumer_timeout_ms=120000
        )

        consumer.subscribe(['activate-layer'])
        print(f"Layer {self.layer_id} waiting for activation message...")

        for message in consumer:
            if message.value.get('layer') == self.layer_id:
                print(f"Layer {self.layer_id} activated with message: {message.value}")
                consumer.commit()
                break

        consumer.close()

        # Clear old messages in activate-layer topic
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BROKER,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        #producer.send('activate-layer', {'layer': None})  # Dummy message to clear old activations

        # Activate neurons via Kafka
        topic = f'layer-{self.layer_id[-1]}'
        activation_message = {'layer': self.layer_id}

        for neuron_id in range(len(self.neurons)):
            producer.send(topic, key=str(neuron_id).encode(), value=activation_message)
            print(f"Neuron {neuron_id} in {topic} activated")

        outputs = np.array([neuron.forward(input_data) for neuron in self.neurons])
        if self.is_final_layer:
            outputs = softmax(outputs)

        redis_handler.set(self.layer_id, outputs.astype(np.float32).tobytes())

        # Activate the next layer
        if not self.is_final_layer:
            next_layer = f'layer-{int(self.layer_id[-1]) + 1}'
            print(f"Activating next layer: {next_layer}")
            producer.send('activate-layer', {'layer': next_layer})

        producer.flush()
        producer.close()

        return outputs

def forward_pass(layers, input_data, image_id):
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BROKER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    print("🔥 Sending initial activation message to activate-layer topic...")
    producer.send('activate-layer', {'layer': layers[0].layer_id})
    producer.flush()
    producer.close()

    for layer in layers:
        input_data = layer.forward(input_data)

    prediction = int(np.argmax(input_data))
    redis_handler.hset('predictions', image_id, prediction)
    return prediction


# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Build network
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [
            Neuron(
                layer_id=layer_name,
                neuron_id=idx,
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            for idx, node in enumerate(layer_info['nodes'])
        ]
        layers.append(Layer(layer_id=layer_name, neurons=neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers
        
# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    prediction = forward_pass(network, image_np, i)
    print(f"Image {i} Prediction: {prediction}, Label: {label}")

# Calculate accuracy
predictions = redis_handler.hgetall('predictions')
correct = sum(int(predictions[k]) == mnist_test[int(k)][1] for k in predictions)
accuracy = correct / len(predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


🔥 Sending initial activation message to activate-layer topic...
Layer layer_0 waiting for activation message...
Neuron 0 in layer-0 activated
Neuron 1 in layer-0 activated
Neuron 2 in layer-0 activated
Neuron 3 in layer-0 activated
Neuron 4 in layer-0 activated
Neuron 5 in layer-0 activated
Neuron 6 in layer-0 activated
Neuron 7 in layer-0 activated
Neuron 8 in layer-0 activated
Neuron 9 in layer-0 activated
Neuron 10 in layer-0 activated
Neuron 11 in layer-0 activated
Neuron 12 in layer-0 activated
Neuron 13 in layer-0 activated
Neuron 14 in layer-0 activated
Neuron 15 in layer-0 activated
Neuron 16 in layer-0 activated
Neuron 17 in layer-0 activated
Neuron 18 in layer-0 activated
Neuron 19 in layer-0 activated
Neuron 20 in layer-0 activated
Neuron 21 in layer-0 activated
Neuron 22 in layer-0 activated
Neuron 23 in layer-0 activated
Neuron 24 in layer-0 activated
Neuron 25 in layer-0 activated
Neuron 26 in layer-0 activated
Neuron 27 in layer-0 activated
Neuron 28 in layer-0 activated

KeyboardInterrupt: 